<a href="https://colab.research.google.com/github/Noufal21/Lightgbm-automl/blob/main/feature-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tsflex tsfresh powershap
#!pip install pycatch22
!pip install  tsai

In [ ]:
!pip install swifter
!pip install ray

In [2]:
import numpy as np
import pandas as pd
#from tsflex.features.integrations import tsfresh_settings_wrapper, catch22_wrapper
#import pycatch22
import swifter
import ray

In [3]:

X_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/X_train_1.park")
y_train_a = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/y_train_1.park")
knowledge = pd.read_parquet("https://storage.googleapis.com/public-quant/bankrupt_2022/knowledge.park")

X_train_a["ticker"] = knowledge["ticker"]
X_train_a["date"] = knowledge["date"]

In [4]:
X_train_a.columns

Index(['return_mean_monthly', 'log_return_mean_monthly',
       'volume_pct_mean_monthly', 'log_volume_pct_mean_monthly',
       'log_return_7_mean_mean_monthly', 'log_return_7_std_mean_monthly',
       'log_volume_pct_7_mean_mean_monthly',
       'log_volume_pct_7_std_mean_monthly', 'return_std_monthly',
       'log_return_std_monthly',
       ...
       'sps', 'tangibles', 'taxassets', 'taxexp', 'taxliabilities', 'tbvps',
       'workingcapital', 'delay', 'ticker', 'date'],
      dtype='object', length=123)

In [ ]:
from numba import njit
import numpy as np
import multiprocessing as mp

## get more functions at above link

def _CO_trev_1_num(X):
    # Time-reversibility statistic, ((x_t+1 − x_t)^3)_t.
    y = np.zeros(len(X) - 1)
    for i in range(len(y)):
        y[i] = np.power(X[i + 1] - X[i], 3)
    return np.mean(y)

def _MD_hrv_classic_pnn40(X):
    # Proportion of successive differences exceeding 0.04σ (Mietus 2002).
    diffs = np.zeros(len(X) - 1)
    for i in range(len(diffs)):
        diffs[i] = np.abs(X[i + 1] - X[i]) * 1000

    nsum = 0
    for diff in diffs:
        if diff > 40:
            nsum += 1

    return nsum / len(diffs)

    
arguments = [(12, _CO_trev_1_num,"_CO_trev_1_num"),(12, _MD_hrv_classic_pnn40,"_MD_hrv_classic_pnn40")]
def get_rolling(arg):
    window, func, func_name  = arg
    return X_train_a[["log_return_mean_monthly","return_mean_monthly","ticker"]].groupby(["ticker"]).rolling(window).apply(func,engine='numba',raw=True).add_suffix("{}{}".format(str(window),func_name))

pool = mp.Pool(processes = (mp.cpu_count() - 1))
results = pool.map(get_rolling, arguments)
pool.close()
pool.join()
results_df = pd.concat(results)

In [25]:

import numba
## get more functions at above link
@numba.jit(nopython=True,cache=False)
def _CO_trev_1_num(X):
    # Time-reversibility statistic, ((x_t+1 − x_t)^3)_t.
    y = np.zeros(len(X) - 1)
    for i in range(len(y)):
        y[i] = np.power(X[i + 1] - X[i], 3)
    return np.mean(y)


@numba.jit(nopython=True,cache=False)  
def _MD_hrv_classic_pnn40(X):
    # Proportion of successive differences exceeding 0.04σ (Mietus 2002).
    diffs = np.zeros(len(X) - 1)
    for i in range(len(diffs)):
        diffs[i] = np.abs(X[i + 1] - X[i]) * 1000

    nsum = 0
    for diff in diffs:
        if diff > 40:
            nsum += 1

    return nsum / len(diffs)


### Testing function using original function provided by lib

In [9]:
%%timeit
X_train_a[["log_return_mean_monthly","ticker"]].groupby(["ticker"]).rolling(12).apply(_CO_trev_1_num,engine='numba',
                                                                                      engine_kwargs = {'nopython': True, 'nogil': False},
                                                                                      raw=True)

4.4 s ± 60.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [88]:
%%timeit
X_train_a[["log_return_mean_monthly","ticker"]].groupby(["ticker"]).rolling(12).apply(_MD_hrv_classic_pnn40,engine='numba',
                                                                                      engine_kwargs = {'nopython': True, 'nogil': False},
                                                                                      raw=True)

2.91 s ± 75.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Testing same function by trasnforming into numpy

In [58]:
def COtrev_1_num_numpy(X):
  return np.mean(np.power(np.diff(X),3))



def MDhrv_classic_pnn40_numpy(X):
  diffs = np.abs(np.diff(X)) * 1000
  return np.sum(diffs > 40)/len(diffs)

In [86]:
%%timeit
X_train_a[["log_return_mean_monthly","ticker"]].groupby(["ticker"]).rolling(12).apply(COtrev_1_num_numpy,engine='numba',
                                                                                      engine_kwargs = {'nopython': True, 'nogil': False},
                                                                                      raw=True)

4.14 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [87]:
%%timeit
X_train_a[["log_return_mean_monthly","ticker"]].groupby(["ticker"]).rolling(12).apply(MDhrv_classic_pnn40_numpy,engine='numba',
                                                                                      engine_kwargs = {'nopython': True, 'nogil': False},
                                                                                      raw=True)

3.51 s ± 46.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


From the above experiment the original function are faster than the numpy function. The Input type of these function are panda.series

### In this expermient I tried to vectorize the group by to increase the performance

In [61]:
def cust_groupby(tickers_ids, feature_value):
  unique_ticker_id  ,indices= np.unique(tickers_ids,return_inverse=True)
  d = np.diff(np.concatenate([[-np.inf], indices,[np.inf]]))
  inds = np.where(d!=0)[0]

  groupByvalue = {
      uli:feature_value[inds[ix]: inds[ix+1]] for ix,uli in enumerate(unique_ticker_id) 
  }
  return groupByvalue

In [62]:
%%timeit
groupByvalue = cust_groupby(X_train_a['ticker'].values,X_train_a['log_return_mean_monthly'].values)

732 ms ± 6.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%%timeit
X_train_a[["log_return_mean_monthly","ticker"]].groupby(["ticker"]).rolling(12)

21.6 ms ± 286 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Pandas groupby is faster. I am not convinced from my result. I am doing something wrong in the groupby function. I am going to spend more time to see how to optimize groupbyrolling in a better way or the original function given by pandas are the optimize one's.



### Trying to find a way to run multiple function in one lambda. 

As doing this through miltiporcessing taking to much time. As opening the threads are always more expensive than the work in our case.

In [47]:
def f(X):
    #d = {}
    print(X.index[-1])
    a = _CO_trev_1_num(X)
    b = _MD_hrv_classic_pnn40(X)
    b_wi_nb.append([a,b])
    return 0#pd.Series(d, index=['12__CO_trev_1_num', '12__MD_hrv_classic_pnn40'])

In [48]:
from numba.typed import List
b_wi_nb = List()

In [49]:
%%timeit
X_train_a[["log_return_mean_monthly","ticker"]].groupby(["ticker"]).rolling(12).apply(f,engine='numba',
                                                                                      engine_kwargs = {'nopython': True, 'nogil': False,'parallel' : True},
                                                                                      raw=True)

TypingError: ignored